In [6]:
from datasets import load_dataset
from datasets import load_dataset
train_dataset = load_dataset('json', data_files='/Users/sam/Desktop/finetune-llm-for-rag/datasets/sql/MiniLM-L6/default_ef_samlhuillier-sql-create-context-spider-intersect-train-with-1-examples.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='/Users/sam/Desktop/finetune-llm-for-rag/datasets/sql/MiniLM-L6/default_ef_samlhuillier-sql-create-context-spider-intersect-validation-with-1-examples.jsonl', split='train')
print(train_dataset)
print(eval_dataset)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 202.26it/s]
Generating train split: 3961 examples [00:00, 82242.88 examples/s]
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 292.73it/s]
Generating train split: 568 examples [00:00, 82677.93 examples/s]

Dataset({
    features: ['db_id', 'context', 'question', 'answer', 'full_prompt', 'inference_prompt'],
    num_rows: 3961
})
Dataset({
    features: ['db_id', 'context', 'question', 'answer', 'full_prompt', 'inference_prompt'],
    num_rows: 568
})


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

# %%
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [8]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        # truncation=True,
        # max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

# %% [markdown]
# And run convert each data_point into a prompt that I found online that works quite well:

# %%
def generate_and_tokenize_prompt(data_point):
    full_prompt = data_point["full_prompt"]
    # print(full_prompt)
    tokenized = tokenize(full_prompt)
    # print("length of prompt: ", len(full_prompt))
    length_of_tokenized_prompt = len(tokenized["input_ids"])
    if length_of_tokenized_prompt > max_length:
        max_length = length_of_tokenized_prompt
    # print("length of tokenized prompt: ", len(tokenized["input_ids"]))
    return tokenized
# print(max_length)

In [9]:
max_length = 0
for data_point in train_dataset:
    full_prompt = data_point["full_prompt"]
    # print(full_prompt)
    tokenized = tokenize(full_prompt)
    # print("length of prompt: ", len(full_prompt))
    length_of_tokenized_prompt = len(tokenized["input_ids"])
    if length_of_tokenized_prompt > max_length:
        max_length = length_of_tokenized_prompt
print(max_length)

544


In [10]:
max_length = 0
for data_point in eval_dataset:
    full_prompt = data_point["full_prompt"]
    # print(full_prompt)
    tokenized = tokenize(full_prompt)
    # print("length of prompt: ", len(full_prompt))
    length_of_tokenized_prompt = len(tokenized["input_ids"])
    if length_of_tokenized_prompt > max_length:
        max_length = length_of_tokenized_prompt
print(max_length)

444


In [14]:
# tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/568 [00:00<?, ? examples/s]

UnboundLocalError: local variable 'max_length' referenced before assignment